# Area source: compare earthquake occurrence from seismicity and geology

What's the relationship between the earthquake occurrence predicted by looking at past seismicity and earthquake occurrence derived from geological information? Here we compare the magnitude-frequency distribution assigned to an area source against the observed seismicity and the magnitude-frequency distribution (MFD) obtained by summing the MFDs of the faults whose trace intersects the polygon of the area source.

In [ ]:
%matplotlib inline
import os
import re
import sys
import h5py
import numpy
import matplotlib.pyplot as plt
import matplotlib.patheffects as PathEffects
from matplotlib import rc
from oqmbt.tools.utils import GetSourceIDs
from oqmbt.oqt_project import OQtProject
from oqmbt.tools.mfd import EEvenlyDiscretizedMFD 

## Info

In [ ]:
project_pickle_filename = os.environ.get('OQMBT_PROJECT')
print('Reading project file: %s' % (project_pickle_filename))
oqtkp = OQtProject.load_from_file(project_pickle_filename)
model_id = oqtkp.active_model_id
model = oqtkp.models[model_id]
eqk_rates_hdf5_filename = os.path.join(oqtkp.directory, oqtkp.eqk_rates_hdf5_filename)
try:
    key = getattr(oqtkp,'active_source_id')[0]
    print('Processing area source with ID: ', key)
except:
    print('Active source ID not defined in the OQMBT project')

In [ ]:
def get_cumulative(mfd):
    """
    Compute a cumulative MFD from a (discrete) incremental one
    
    :param mfd:
        An :class:'openquake.hazardlib.mfd.BaseMFD' instance
    :returns: 
        Two lists, the first one containing magnitudes values and the 
        second one with annual rates of exceedance (m>m0).
     """
    mags = []
    cml  = []
    occs = []
    #
    # loading information for the original MFD
    for mag, occ in mfd.get_annual_occurrence_rates():
        mags.append(mag)
        occs.append(occ)
    #
    # reverting rates
    for occ in reversed(occs):
        if len(cml):
            cml.append(occ+cml[-1])
        else:
            cml.append(occ)
    #
    return mags, cml[::-1]

## Results

In [ ]:
#
# min and max values for plotting
rate_min = 1e-5
rate_max = 1e1
mag_min = 4.5
mag_max = 9.0
#
# loading source
src = model.sources[key]
#
# printing info
print 'Area source ID   : %s' % (src.source_id)
print 'Area source name : %s' % (src.name)
if hasattr(src, 'tectonic_region_type'):
    print 'Tectonic region  : %s' % (src.tectonic_region_type)
#
# initialize some lists
labs = []
xlab = []
ylab = []
ylabc = []
#
# initialize the plot
fig = plt.figure(1, figsize=(10,8))
ax = fig.add_axes([0.1,0.1,0.8,0.8])
ax.set_title('Area source:' + key, fontsize=16)
#
fig = plt.figure(2, figsize=(10,8))
ax2 = fig.add_axes([0.1,0.1,0.8,0.8])
ax2.set_title('Area source:' + key, fontsize=16)
#
# Get seismicity rates for the selected area source
fhdf5 = h5py.File(eqk_rates_hdf5_filename,'r')
rates = fhdf5[model_id][key][:]
fhdf5.close()
#
stack = None
#
#
if hasattr(src, 'ids_faults_inside'):
    #
    # IDs of faults inside the area source
    print(src.ids_faults_inside)
    for iii, aaa in enumerate(sorted(src.ids_faults_inside.keys())):  
        tmp_src = model.sources[aaa] 
        
        if hasattr(tmp_src, 'mfd') and tmp_src.mfd is not None:
            ratio = src.ids_faults_inside[aaa]

            tmfd = tmp_src.mfd
            tmfd.occurrence_rates = list(numpy.array(tmfd.occurrence_rates) * ratio)
            srates = tmfd.occurrence_rates

            mags = tmfd.min_mag + numpy.arange(0, len(srates), 1.)*tmfd.bin_width
            macm, occm = get_cumulative(tmfd)
            
            plt.figure(1)
            plt.plot(mags, srates, '-y')
            plt.figure(2)
            plt.plot(macm, occm, '-y')
            
            if (srates[0] > rate_min):
                labs.append(aaa)
                xlab.append(mags[0])
                ylab.append(srates[0])
                
                ylabc.append(occm[0])

            if stack is None :
                stack = EEvenlyDiscretizedMFD(tmfd.min_mag, tmfd.bin_width, tmfd.occurrence_rates) 
            else:
                stack.stack(tmfd)

    if stack is not None:
        mags = stack.min_mag + numpy.arange(0, len(stack.occurrence_rates), 1.)*stack.bin_width
        
        plt.figure(1)
        plt.plot(mags, stack.occurrence_rates, '.-b')
       
        plt.figure(2)
        macm, occm = get_cumulative(stack)
        plt.plot(macm, occm, '.-b') 
        
        del stack
        
plt.figure(1)

# Create labels
effect = [PathEffects.withStroke(linewidth=5, foreground="LightYellow")]
num = len(labs)+2

lab_y = numpy.logspace(numpy.log10(rate_min), numpy.log10(rate_max),num)
sindexes = numpy.argsort(numpy.array(ylab))
sidxc = numpy.argsort(numpy.array(ylabc))

for prog, iii in enumerate(sindexes):
    plt.figure(1)
    plt.text(mag_min+0.1, lab_y[prog+1], labs[iii], path_effects=effect)
    plt.plot([mag_min+0.1, xlab[iii]], [lab_y[prog+1], ylab[iii]], ':', 
             color='blue', linewidth=0.5)   

for prog, iii in enumerate(sidxc):
    plt.figure(2)
    plt.text(mag_min+0.1, lab_y[prog+1], labs[iii], path_effects=effect)
    plt.plot([mag_min+0.1, xlab[iii]], [lab_y[prog+1], ylabc[iii]], ':', 
             color='blue', linewidth=0.5) 
    
# Plot observed seismicity 
rat = rates[2,:]/rates[1,:]
ax.plot(rates[0,:], rat, 'or')

plt.figure(2)
xxx = numpy.cumsum(rat[::-1])
ax2.plot(rates[0,:], xxx[::-1], 'or')

if hasattr(src, 'mfd'):
    #
    # getting info from the original MFD
    tmfd = src.mfd
    mags = tmfd.min_mag + numpy.arange(0, len(tmfd.occurrence_rates), 1.)*tmfd.bin_width
    #
    # figure 1 plotting
    plt.figure(1)
    plt.plot(mags, tmfd.occurrence_rates, 'r--')
    plt.text(mag_max-0.8, rate_max-rate_max*0.5, 'b_gr: %.2f' % src.b_gr, 
         path_effects=effect)
    macm, occm = get_cumulative(tmfd)
    # figure 2
    plt.figure(2)
    plt.plot(macm, occm, 'r--')
    #
    # figure 1
    plt.figure(1)
    ax.set_ylim([rate_min, rate_max])
    ax.set_xlim([mag_min, mag_max])
    ax.set_yscale('log')
    ax.set_xlabel('Magnitude, m []', fontsize=16)
    ax.set_ylabel(r'Rate, r [eqks/yr]', fontsize=16)    
    ax.grid(which='both')
    plt.tick_params(axis='both', which='major', labelsize=14)
    #
    # figure 2
    plt.figure(2)
    ax2.set_ylim([rate_min, rate_max])
    ax2.set_xlim([mag_min, mag_max])
    ax2.set_yscale('log')
    ax2.set_xlabel('Magnitude, m []', fontsize=16)
    ax2.set_ylabel(r'Rate of exceedance [eqks/yr]', fontsize=16)
    ax2.grid(which='both')
    plt.tick_params(axis='both', which='major', labelsize=14)
    plt.text(mag_max-0.8, rate_max-rate_max*0.5, 'b_gr: %.2f' % src.b_gr, 
         path_effects=effect)
else:
    print 'Source %s does not have an MFD' % (src.source_id)